### Kombinasi 2 :
* Delete Duplicate
* Impute Null pake iterative imputer
* Outlier capping di sisi kanan
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [72]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [73]:
df = pd.read_csv('../UFC_train.csv')

df.head()

,R_fighter,B_fighter,Referee,date,location,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,Winner
0,Joe Riggs,Joe Doerksen,Steve Mazzagatti,2004-08-21,"Las Vegas, Nevada, USA",False,Middleweight,NaN,NaN,NaN,...,0,0,0,Southpaw,182.88,177.80,185.0,26.0,21.0,Red
1,Jorge Masvidal,Al Iaquinta,Keith Peterson,2015-04-04,"Fairfax, Virginia, USA",False,Lightweight,1.156250,0.000000,0.394141,...,0,1,0,Orthodox,180.34,187.96,170.0,27.0,30.0,Blue
2,Dan Stittgen,Stephen Thompson,Josh Rosenthal,2012-02-04,"Las Vegas, Nevada, USA",False,Welterweight,NaN,NaN,NaN,...,0,0,0,Orthodox,185.42,NaN,170.0,28.0,31.0,Blue
3,Josh Koscheck,Johny Hendricks,Kevin Mulhall,2012-05-05,"East Rutherford, New Jersey, USA",False,Welterweight,0.695312,0.000000,0.783359,...,6,3,0,Orthodox,177.80,185.42,170.0,28.0,34.0,Blue
4,John Dodson,Manvel Gamburyan,James Warring,2016-04-16,"Tampa, Florida, USA",False,Bantamweight,0.500000,0.266602,0.381462,...,3,0,1,Orthodox,160.02,167.64,135.0,34.0,31.0,Red


In [74]:
# Delete duplicate
print(f"Shape before dropping duplicates : {df.shape}") # Before dropping duplicates
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df.shape}") # After dropping duplicates

Shape before dropping duplicates : (5410, 144)
Shape after dropping duplicates: (5410, 144)


In [75]:
def check_null(df):
    col_na = df.isnull().sum().sort_values(ascending=True)
    percent = col_na / len(df)
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])

    if (missing_data[missing_data['Total'] > 0].shape[0] == 0):
        print("Tidak ditemukan missing value pada dataset")
    else:
        print(missing_data[missing_data['Total'] > 0])

In [76]:
check_null(df)

                         Total   Percent
R_Weight_lbs                 2  0.000370
R_Height_cms                 4  0.000739
B_Weight_lbs                 8  0.001479
B_Height_cms                10  0.001848
R_Stance                    27  0.004991
...                        ...       ...
B_avg_GROUND_landed       1293  0.239002
B_avg_GROUND_att          1293  0.239002
B_avg_opp_CLINCH_landed   1293  0.239002
B_avg_TD_att              1293  0.239002
B_avg_HEAD_landed         1293  0.239002

[109 rows x 2 columns]


In [77]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [78]:
# Impute missing value using iterative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer()
df_imputed_number = imputer.fit_transform(df_number)
df_imputed_number = pd.DataFrame(df_imputed_number, columns=df_number.columns)

c:\Python311\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [79]:
# Drop R_fighter, B_fighter, Referee, location, and date
df_object = df_object.drop(['R_fighter', 'B_fighter', 'Referee', 'location', 'date'], axis=1)

In [80]:
# Combine df_object and df_imputed_number
df = pd.concat([df_object, df_imputed_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,-1.558000,-269.960915,474.696197,1.001642,0.286333,3.761259,...,0.0,0.0,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,1.156250,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,-0.169817,-59.257790,104.407134,0.565607,0.253130,1.065947,...,0.0,0.0,0.0,0.0,0.0,185.42,177.846319,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.783359,0.185547,0.088281,0.104375,...,1.0,5.0,6.0,3.0,0.0,177.80,185.420000,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,1.0,160.02,167.640000,135.0,34.0,31.0


In [81]:
check_null(df)

          Total   Percent
R_Stance     27  0.004991
B_Stance     64  0.011830


In [82]:
# Impute missing R_stance and B_stance with mode
df['R_Stance'].fillna(df['R_Stance'].mode()[0], inplace=True)
df['B_Stance'].fillna(df['B_Stance'].mode()[0], inplace=True)

In [83]:
check_null(df)

Tidak ditemukan missing value pada dataset


In [84]:
def check_outlier(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)

    # Menghitung RUB dan RLB.
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    # Menampilkan banyaknya outlier pada atribut.
    outliers = (df < lower_limit) | (df > upper_limit)
    print ("Outlier pada tiap atribut:")
    print(outliers.sum())

    return outliers

In [85]:
# Split string and number
# Split dataframe into object type and number type
df_object = df.select_dtypes(include='object')
df_number = df.select_dtypes(include=np.number)

In [86]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

Outlier pada tiap atribut:
B_avg_KD                  519
B_avg_opp_KD             1292
B_avg_SIG_STR_pct        1311
B_avg_opp_SIG_STR_pct     323
B_avg_TD_pct              186
                         ... 
R_Height_cms               19
R_Reach_cms               106
R_Weight_lbs              301
B_age                      38
R_age                      98
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 9.593345656192238%
B_avg_opp_KD = 23.88170055452865%
B_avg_SIG_STR_pct = 24.23290203327172%
B_avg_opp_SIG_STR_pct = 5.970425138632162%
B_avg_TD_pct = 3.4380776340110906%
B_avg_opp_TD_pct = 8.484288354898336%
B_avg_SUB_ATT = 9.353049907578558%
B_avg_opp_SUB_ATT = 11.44177449168207%
B_avg_REV = 12.513863216266174%
B_avg_opp_REV = 40.46210720887246%
B_avg_SIG_STR_att = 4.306839186691312%
B_avg_SIG_STR_landed = 6.7467652495378925%
B_avg_opp_SIG_STR_att = 4.953789279112754%
B_avg_opp_SIG_STR_landed = 5.138632162661738%
B_avg_TOTAL_STR_att = 3.5674676524953792%
B_a

In [87]:
from scipy.stats.mstats import winsorize

# Identify columns with more than 0.5% outliers
outlier_columns = []
for key in outliers.keys():
    if outliers[key] / df_number[key].shape[0] > 0.005:
        outlier_columns.append(key)

# Winsorize the identified columns
for column in outlier_columns:
    df_number[column] = winsorize(df_number[column], limits=(0, 0.1))

# Combine df_object and df_number
df = pd.concat([df_object, df_number], axis=1)
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Middleweight,Orthodox,Southpaw,Red,-1.558000,-269.960915,0.621484,0.585000,0.286333,0.660000,...,0.0,0.0,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0
1,Lightweight,Orthodox,Orthodox,Blue,0.750000,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,4.0,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0
2,Welterweight,Orthodox,Orthodox,Blue,-0.169817,-59.257790,0.621484,0.565607,0.253130,0.660000,...,0.0,0.0,0.0,0.0,0.0,185.42,177.846319,170.0,28.0,31.0
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.621484,0.185547,0.088281,0.104375,...,1.0,4.0,4.0,2.0,0.0,177.80,185.420000,170.0,28.0,34.0
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,0.0,2.0,3.0,0.0,0.0,160.02,167.640000,135.0,34.0,31.0


In [88]:
outliers = dict(check_outlier(df_number).sum())
print("\n\npercentage of outliers in each column:")
for key in outliers.keys():
    print(f"{key} = {outliers[key]/df_number[key].shape[0] * 100}%")

c:\Python311\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


Outlier pada tiap atribut:
B_avg_KD                   62
B_avg_opp_KD             1292
B_avg_SIG_STR_pct        1091
B_avg_opp_SIG_STR_pct     119
B_avg_TD_pct                0
                         ... 
R_Height_cms               19
R_Reach_cms                53
R_Weight_lbs                0
B_age                       4
R_age                      12
Length: 134, dtype: int64


percentage of outliers in each column:
B_avg_KD = 1.146025878003697%
B_avg_opp_KD = 23.88170055452865%
B_avg_SIG_STR_pct = 20.166358595194083%
B_avg_opp_SIG_STR_pct = 2.1996303142329023%
B_avg_TD_pct = 0.0%
B_avg_opp_TD_pct = 2.236598890942699%
B_avg_SUB_ATT = 1.8114602587800372%
B_avg_opp_SUB_ATT = 2.6432532347504623%
B_avg_REV = 12.513863216266174%
B_avg_opp_REV = 40.46210720887246%
B_avg_SIG_STR_att = 0.03696857670979667%
B_avg_SIG_STR_landed = 3.4011090573012934%
B_avg_opp_SIG_STR_att = 0.018484288354898334%
B_avg_opp_SIG_STR_landed = 0.0%
B_avg_TOTAL_STR_att = 0.07393715341959334%
B_avg_TOTAL_STR_landed

c:\Python311\Lib\site-packages\numpy\lib\function_base.py:4823: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


In [89]:
df['R_Gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')
df['B_Gender'] = df['weight_class'].apply(lambda x: 'women' if 'women' in x.lower() else 'male')


In [90]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_Gender,B_Gender
0,Middleweight,Orthodox,Southpaw,Red,-1.558000,-269.960915,0.621484,0.585000,0.286333,0.660000,...,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0,male,male
1,Lightweight,Orthodox,Orthodox,Blue,0.750000,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0,male,male
2,Welterweight,Orthodox,Orthodox,Blue,-0.169817,-59.257790,0.621484,0.565607,0.253130,0.660000,...,0.0,0.0,0.0,185.42,177.846319,170.0,28.0,31.0,male,male
3,Welterweight,Southpaw,Orthodox,Blue,0.695312,0.000000,0.621484,0.185547,0.088281,0.104375,...,4.0,2.0,0.0,177.80,185.420000,170.0,28.0,34.0,male,male
4,Bantamweight,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,3.0,0.0,0.0,160.02,167.640000,135.0,34.0,31.0,male,male


In [91]:
df['weight_class'] = df['weight_class'].str.replace('Women', '')

In [92]:
# Get all weight_class values
weight_class = df['weight_class'].unique()
weight_class

array(['Middleweight', 'Lightweight', 'Welterweight', 'Bantamweight',
       'Flyweight', 'LightHeavyweight', 'Strawweight', 'Featherweight',
       'OpenWeight', 'Heavyweight', 'CatchWeight'], dtype=object)

In [93]:
# Create a dictionary to map weight_class values to numbers
weight_class_dict = {
    'CatchWeight' : 0,
    'Strawweight' : 1,
    'Flyweight' : 2,
    'Bantamweight' : 3,
    'Featherweight' : 4,
    'Lightweight' : 5,
    'Welterweight' : 6,
    'Middleweight' : 7,
    'LightHeavyweight' : 8,
    'Heavyweight' : 9,
    'OpenWeight' : 10,
}

gender_dict = {
    'male' : 1,
    'women' : 0
}

In [94]:
# Map each weight_class value to the correct number
df['weight_class'] = df['weight_class'].map(weight_class_dict)

In [95]:
# Map each gender value to the correct number
df['R_Gender'] = df['R_Gender'].map(gender_dict)
df['B_Gender'] = df['B_Gender'].map(gender_dict)

In [96]:
df.head()

,weight_class,B_Stance,R_Stance,Winner,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_Gender,B_Gender
0,7,Orthodox,Southpaw,Red,-1.558000,-269.960915,0.621484,0.585000,0.286333,0.660000,...,0.0,0.0,0.0,182.88,177.800000,185.0,26.0,21.0,1,1
1,5,Orthodox,Orthodox,Blue,0.750000,0.000000,0.394141,0.352422,0.239219,0.011484,...,0.0,1.0,0.0,180.34,187.960000,170.0,27.0,30.0,1,1
2,6,Orthodox,Orthodox,Blue,-0.169817,-59.257790,0.621484,0.565607,0.253130,0.660000,...,0.0,0.0,0.0,185.42,177.846319,170.0,28.0,31.0,1,1
3,6,Southpaw,Orthodox,Blue,0.695312,0.000000,0.621484,0.185547,0.088281,0.104375,...,4.0,2.0,0.0,177.80,185.420000,170.0,28.0,34.0,1,1
4,3,Orthodox,Orthodox,Red,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,...,3.0,0.0,0.0,160.02,167.640000,135.0,34.0,31.0,1,1


In [97]:
# Get the remaining object columns
df_object = df.select_dtypes(include='object')

# Perform one-hot encoding
df_encoded = pd.get_dummies(df_object)

# Combine the encoded columns with the original dataframe
df = pd.concat([df, df_encoded], axis=1)

# Drop the original object columns
df.drop(df_object.columns, axis=1, inplace=True)

df.head()


,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,...,B_Stance_Southpaw,B_Stance_Switch,R_Stance_Open Stance,R_Stance_Orthodox,R_Stance_Sideways,R_Stance_Southpaw,R_Stance_Switch,Winner_Blue,Winner_Draw,Winner_Red
0,7,-1.558000,-269.960915,0.621484,0.585000,0.286333,0.660000,-4.590713,1.125000,0.344289,...,False,False,False,False,False,True,False,False,False,True
1,5,0.750000,0.000000,0.394141,0.352422,0.239219,0.011484,0.156250,0.132812,0.000000,...,False,False,False,True,False,False,False,True,False,False
2,6,-0.169817,-59.257790,0.621484,0.565607,0.253130,0.660000,-0.614151,1.125000,0.201711,...,False,False,False,True,False,False,False,True,False,False
3,6,0.695312,0.000000,0.621484,0.185547,0.088281,0.104375,0.093750,0.062500,0.000000,...,True,False,False,True,False,False,False,True,False,False
4,3,0.500000,0.266602,0.381462,0.456558,0.429614,0.469570,1.377930,0.000244,0.125000,...,False,False,False,True,False,False,False,False,False,True


In [105]:
from sklearn.feature_selection import SelectKBest, f_regression
# Perform feature selection using SelectKBest
selector = SelectKBest(k=20, score_func=f_regression)
X = df.drop('B_Reach_cms', axis=1) 
y = df['B_Reach_cms'] 
X_selected = selector.fit_transform(X, y)

# Get the selected feature names
selected_feature_names = X.columns[selector.get_support()]
print(selected_feature_names)
# Create a new dataframe with only the selected features
df_selected = pd.DataFrame(X_selected, columns=selected_feature_names)
df_selected.head()


Index(['R_avg_KD', 'R_avg_opp_KD', 'R_avg_SIG_STR_pct', 'R_avg_TD_pct',
       'R_avg_SUB_ATT', 'R_avg_REV', 'R_avg_SIG_STR_landed',
       'R_avg_opp_SIG_STR_att', 'R_avg_opp_TOTAL_STR_att',
       'R_avg_opp_TOTAL_STR_landed', 'R_avg_TD_landed', 'R_avg_opp_TD_att',
       'R_avg_opp_TD_landed', 'R_avg_HEAD_landed', 'R_avg_opp_HEAD_att',
       'R_avg_opp_HEAD_landed', 'R_avg_opp_LEG_att',
       'R_avg_opp_DISTANCE_landed', 'R_avg_opp_CLINCH_att',
       'R_avg_GROUND_landed'],
      dtype='object')


,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,R_avg_TD_pct,R_avg_SUB_ATT,R_avg_REV,R_avg_SIG_STR_landed,R_avg_opp_SIG_STR_att,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_avg_TD_landed,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_HEAD_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_LEG_att,R_avg_opp_DISTANCE_landed,R_avg_opp_CLINCH_att,R_avg_GROUND_landed
0,1.000000,1.000000,0.700000,0.875000,2.000000,1.000000,83.250000,181.000000,205.320312,111.320312,4.218750,8.90625,3.750000,55.750000,147.000000,52.496094,21.388426,63.000000,24.000000,21.781250
1,0.500000,0.343750,0.498750,0.554688,0.562500,0.500000,60.562500,117.812500,129.593750,53.093750,1.875000,5.81250,0.937500,37.781250,92.250000,27.656250,7.968750,38.406250,4.562500,7.250000
2,1.000000,1.000000,0.700000,0.875000,2.000000,1.000000,83.250000,181.000000,205.320312,111.320312,4.218750,8.90625,3.750000,55.750000,147.000000,52.496094,21.388426,63.000000,24.000000,21.781250
3,0.020050,0.011723,0.373518,0.419368,0.093866,0.000015,27.139767,109.998363,125.441042,53.259472,1.440809,3.66069,1.000504,19.695154,92.600458,27.688076,8.843031,33.409025,6.983452,4.844223
4,0.320312,0.000000,0.416328,0.466719,0.000000,0.000000,44.851562,126.031250,168.796875,99.320312,1.171875,8.90625,2.156250,15.812500,89.789062,35.625000,12.164062,43.398438,23.234375,0.828125
